# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # Creating a session variable to  establish connection and begin executing queries
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkfy 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkfy')
except Exception as e:
    print(e)

## Functions

In [8]:
def drop_tables(*args):
    """Receive table names and create in cassasandra DB, 
    drop if exists. 
       
       input:
           *args: 1 or more names of tables
       output:
           Void if everything is correct.
    """
    for table_name in args:
        try:
            drop_query = 'DROP TABLE IF EXISTS {}'.format(table_name)
            session.execute(drop_query)
            print("Table {} successfully dropped".format(table_name))
        except Exception as e:
            print(e)

In [9]:
def create_table(table_name, columns_settings):
    """Creates a table with desired columns 
       
       input:
           table_name: string name of table
           columns_settings: string containing columns names and types
       output:
           Void if everything is correct.
    """
    query = f"CREATE TABLE IF NOT EXISTS {table_name}"
    query = query + f"{columns_settings}"
    
    try:
        session.execute(query)
        print("Table {} successfully created".format(table_name))
    except Exception as e:
        print(e)  


In [10]:
def populate_table(table_name, columns, values_positions, dtypes):
    '''
    Pipeline to insert data to Apache Cassandra table from CSV File.
    Input:
        table_name: Name of table to insert data
        columns: string Columns in table to insert
        values_positions: list of indices to pass to INSERT statement
        dtypes: list of values to convert `values` of data to insert
    Output:
        Void if everything is correct
    '''
    
    file = 'event_datafile_new.csv'
    
    assert(len(dtypes) == len(values_positions))
    
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = f"INSERT INTO {table_name} ({columns})" 
            query = query + f" VALUES(%s {(len(dtypes) - 1)*', %s'})"
            data = tuple([ dtype(line[x]) for x, dtype in zip(values, dtypes)])
            session.execute(query, data)

In [13]:
def query_execution(query):
    try:
        result = session.execute(f"{query}")
    except Exception as e:
        print(e)
    for row in result:
        print(row)

def check_inserted_data(query):
    try:
        rows = session.execute(f"{query}")
    except Exception as e:
        print(e)
        
    print("{} rows inserted".format(len(list(rows))))

In [11]:
# CREATE AND POPULATE TABLE
cols = """(session_id int, 
           item_session int, 
           user_id int, 
           first_name text, 
           last_name text, 
           artist text, 
           song text, 
           length decimal, 
           PRIMARY KEY(session_id, item_session))
           """
drop_tables('music_by_session')
create_table('music_by_session', cols) 

cols = "session_id, item_session, user_id, first_name, last_name, artist, song, length"
values = [8,3,10,1,4,0,9,5] #positions for each row
types = [int, int, int, str,str,str,str, float] # types for each value in row

populate_table('music_by_session', cols, values, types)

Table music_by_session successfully dropped
Table music_by_session successfully created


In [14]:
## Select statement to verify if the data was inserted into the table
check_inserted_data("SELECT * FROM music_by_session")

6820 rows inserted


### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during 
###### sessionId = 338, and itemInSession = 4

In [15]:
# Select First Query
query_execution('''SELECT artist, song,length 
                   FROM music_by_session 
                   WHERE session_id = 338 
                   AND item_session = 4''')

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=Decimal('495.3073'))


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [16]:
# CREATE AND POPULATE TABLE
cols = """(user_id int,
           session_id int, 
           item_session int,
           first_name text, 
           last_name text, 
           artist text, 
           song text, 
           PRIMARY KEY(user_id, session_id))
           """
drop_tables('song_by_user_session')
create_table('song_by_user_session', cols) 


cols = "user_id, session_id, item_session, first_name, last_name, artist, song"
values = [10,8,3,1,4,0,9] #positions for each row
types = [int, int, int, str, str, str, str] # types for each value in row

populate_table('song_by_user_session', cols, values, types)

Table song_by_user_session successfully dropped
Table song_by_user_session successfully created


In [17]:
## Select statement to verify if the data was inserted into the table
check_inserted_data("SELECT * FROM song_by_user_session")

776 rows inserted


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
###### for userid = 10, sessionid = 182

In [18]:
# Select Second Query
query_execution('''SELECT artist, song, first_name, last_name 
                   FROM song_by_user_session 
                   WHERE user_id = 10 
                   AND session_id = 182''')

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


In [19]:
# CREATE AND POPULATE TABLE
cols = """(song text,
           user_id int,
           first_name text, 
           last_name text,
           PRIMARY KEY(song, user_id))
           """
drop_tables('user_activity')
create_table('user_activity', cols) 

cols = "song, user_id, first_name, last_name"
values = [9,10,1,4] # positions for each row
types = [str, int, str, str]  # types for each value in row

populate_table('user_activity', cols, values, types)

Table user_activity successfully dropped
Table user_activity successfully created


In [20]:
## Select statement to verify if the data was inserted into the table
check_inserted_data("SELECT * FROM user_activity")

6618 rows inserted


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [21]:
# Select Third Query
query_execution("""SELECT first_name, last_name
                   FROM user_activity 
                   WHERE song = 'All Hands Against His Own'""")

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [23]:
drop_tables('music_by_session', 'song_by_user_session', 'user_activity')

Table music_by_session successfully dropped
Table song_by_user_session successfully dropped
Table user_activity successfully dropped


### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()